In [ ]:
OLLAMA_API = "http://ollama:11434/api/chat"
MODEL = "llama3.2"

headers = {"Content-Type": "application/json"}

# *Let's loop - what is an agent under the hood?

*This section provides a simplified overview of how agents work. 

<img src="https://static.simonwillison.net/static/2025/agents-meme-card.jpg" alt="Agents" width="500">

An **agent** is a system that can interact with an environment by perceiving information and taking actions. 

In the context of LLMs, an agent maintains a conversation history and can make decisions based on previous interactions.

In [87]:
import re
from typing import Literal

from pydantic import BaseModel


class Classification(BaseModel):
    action: Literal["answer", "thinkning", "use_tool"]
    content: str


def classify_response(response_text: str) -> Classification | None:
    response_text = response_text.strip()
    
    tool_match = re.search(fr'USE[_| ]TOOL:\s*calculator\((.*?)\)', response_text, re.DOTALL)
    if tool_match:
        expression = tool_match.group(1).strip()
        return Classification(action="use_tool", content=expression)
    
    answer_match = re.search(r'ANSWER:\s*(.*)', response_text, re.DOTALL)
    if answer_match:
        answer = answer_match.group(1).strip()
        return Classification(action="answer", content=answer)
    
    return None

def calculator(expression: str) -> str:
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {e}"

In [ ]:
import requests
import json

SYSTEM_PROMPT = """You are a helpful assistant with access to a calculator tool.

IMPORTANT RULES:
1. You can ONLY respond with ONE of these formats per message:
   - USE_TOOL: calculator(expression) - when you need to calculate
   - ANSWER: your final answer - when you have the complete answer

2. DO NOT calculate in your head. You MUST use the calculator tool for ANY math operation.
3. DO NOT write multiple USE_TOOL lines in one response. Only ONE per message.
4. DO NOT show the expected result. Just request the tool and wait for the result.
5. Break down complex calculations into SIMPLE individual steps - ONE operation at a time.
6. After getting a calculator result, decide if you need another calculation or can give the final answer.

Example:
User: Calculate (5 + 3) * 2
Assistant: USE_TOOL: calculator(5 + 3)
System: Calculator result: 8
Assistant: USE_TOOL: calculator(8 * 2)
System: Calculator result: 16
Assistant: ANSWER: The result is 16"""

messages = [{"role": "system", "content": SYSTEM_PROMPT}]

user_question = "Calculate: (15 * 7 + 23) * 2 - 100"
messages.append({"role": "user", "content": user_question})
print(f"User: {user_question}\n")

MAX_TURNS = 15

for turn in range(MAX_TURNS):
    print(f"{'='*80}")
    print(f"LOOP ITERATION {turn + 1}")
    print(f"{'='*80}")
    
    response = requests.post(
        url=OLLAMA_API,
        headers=headers,
        data=json.dumps({
            "model": MODEL,
            "messages": messages,
            "stream": False
        })
    )
    
    assistant_content = response.json()["message"]["content"]
    messages.append({"role": "assistant", "content": assistant_content})
    classification = classify_response(assistant_content)
    print(f"🤖 Model says: {assistant_content}\n")
    

    if classification is None:
        print("⚠️  Model didn't use correct format. Prompting again...\n")
        messages.append({
            "role": "assitant",
            "content": "I need to respond with either 'USE_TOOL: calculator(expression)' or 'ANSWER: your answer'"
        })
        continue
    
    if classification.action == "use_tool":
        expression = classification.content
        print(f"🔧 Executing: calculator({expression})")
        
        result = calculator(expression)
        print(f"✅ Result: {result}\n")
        
        messages.append({
            "role": "tool",
            "content": f"{result}\n"
        })
    
    elif classification.action == "answer":
        print(f"✓ Final answer: {classification.content}\n")
        break

User: Calculate: (15 * 7 + 23) * 2 - 100

LOOP ITERATION 1
🤖 Model says: USE_TOOL: calculator(15 * 7)

🔧 Executing: calculator(15 * 7)
✅ Result: 105

LOOP ITERATION 2
🤖 Model says: USE_TOOL: calculator(105 + 23)

🔧 Executing: calculator(105 + 23)
✅ Result: 128

LOOP ITERATION 3
🤖 Model says: USE TOOL: calculator(128 * 2)

🔧 Executing: calculator(128 * 2)
✅ Result: 256

LOOP ITERATION 4
🤖 Model says: USE_TOOL: calculator(256 - 100)

🔧 Executing: calculator(256 - 100)
✅ Result: 156

LOOP ITERATION 5
🤖 Model says: ANSWER: The result is 156

✓ Final answer: The result is 156

